# NHANES Analysis of Anemia Variables - Part 1

In this notebook, we will analyze the correlation among variables and profiles of individuals examined in NHANES survey. The focus is on variables related to anemia diagnosis (see below). This work is based on the research of Patrícia Raia Nogueira Cavoto.

## Selecting the driver
The sentence below defines the driver for the database.

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Reference values for NHANES for the 2005-2006 survey

* Extracted from data of the NHANES Web site (https://wwwn.cdc.gov/nchs/nhanes/).

## Importing normal ranges of values indicated in the NHANES documentation

The following query imports the file `reference-ranges.csv` that contains reference ranges from NHANES.

* For each variable it is indicated
  - applicable gender
  - age range (ageStart until ageEnd)

* The range is indicated in the form of mininum and maximum values considered normal.

In [2]:
DROP TABLE IF EXISTS ReferenceRanges;
CREATE TABLE ReferenceRanges (
  variable VARCHAR(8),
  gender VARCHAR(1),
  ageStart SMALLINT,
  ageEnd SMALLINT,
  min DECIMAL(7,1),
  max DECIMAL(7,1),
  PRIMARY KEY(variable,gender,ageStart,ageEnd)
) AS SELECT
  variable,gender,ageStart,ageEnd,min,max
FROM CSVREAD('../data/nhanes2005-2006/reference-ranges.csv');

SELECT DISTINCT variable FROM ReferenceRanges;
SELECT * FROM ReferenceRanges;

# Survey NHANES 2005-2006

* Extracted from data of the NHANES Web site (https://wwwn.cdc.gov/nchs/nhanes/).

## Importing data from the survey NHANES 2005-2006

* The following query imports the `combined-selected-variables.csv` that contains a tuple for each individual, with a selected set of variables that are used do diagnose anemia, as mentioned in Figure 1. It was filtered only the individuals with values for all fields.

![evaluation of anemia](evaluation-of-anemia.gif "Figure 1")
*Figure 1*: Evaluation of anemia in the adult according to the mean corpuscular volume. CBC: complete blood count; MCV: mean corpuscular volume; RBCs: red blood cells; Fe: iron; TIBC: total iron-binding capacity (transferrin); LDH: lactate dehydrogenase [6].

In [3]:
DROP TABLE IF EXISTS Survey;
CREATE TABLE Survey (
  SEQN VARCHAR(8),
  RIAGENDR VARCHAR(1),
  RIDAGEYR SMALLINT,
  LBXIRN DECIMAL(7,1),
  LBXTIB DECIMAL(7,1),
  LBXSLDSI DECIMAL(7,1),
  LBXWBCSI DECIMAL(7,1),
  LBXLYPCT DECIMAL(7,1),
  LBXMOPCT DECIMAL(7,1),
  LBXNEPCT DECIMAL(7,1),
  LBXEOPCT DECIMAL(7,1),
  LBXBAPCT DECIMAL(7,1),
  LBXRBCSI DECIMAL(7,1),
  LBXHGB DECIMAL(7,1),
  LBXHCT DECIMAL(7,1),
  LBXMCVSI DECIMAL(7,1),
  LBXMCHSI DECIMAL(7,1),
  LBXMC DECIMAL(7,1),
  LBXRDW DECIMAL(7,1),
  LBXPLTSI DECIMAL(7,1),
  LBXMPSI DECIMAL(7,1),
  PRIMARY KEY(SEQN)
) AS SELECT
  SEQN,RIAGENDR,RIDAGEYR,LBXIRN,LBXTIB,LBXSLDSI,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXMC,LBXRDW,LBXPLTSI,LBXMPSI
FROM CSVREAD('../data/nhanes2005-2006/combined-selected-variables.csv');

SELECT COUNT(*) FROM Survey;
SELECT * FROM Survey;

# Codes and description of NHANES variables

* The following query imports the `reference-ranges-variables.csv` that contains codes and description of the variables adopted in this study.

In [4]:
DROP TABLE IF EXISTS VariableDescription;
CREATE TABLE VariableDescription (
  variable VARCHAR(8),
  acronym VARCHAR(8),
  name VARCHAR(50),
  unit VARCHAR(30),
  file VARCHAR(20),
  ranges VARCHAR(100),
  PRIMARY KEY(variable)
) AS SELECT
  variable,acronym,name,unit,file,ranges
FROM CSVREAD('../data/nhanes2005-2006/reference-ranges-variables.csv');

SELECT * FROM VariableDescription;

# Binary evaluation of individuals out of the normal ranges

* For each variable, this table defines an extra binary column _b which is initialized with 0 and will receive 1 is the variable is out of the NHANES range.

## Generation of the starting matrix initialized with 0

In [5]:
DROP TABLE IF EXISTS SurveyB;
CREATE TABLE SurveyB (
  SEQN VARCHAR(8),
  RIAGENDR VARCHAR(1),
  RIDAGEYR SMALLINT,
  LBXIRN DECIMAL(7,1),
  LBXIRN_b SMALLINT DEFAULT 0,
  LBXTIB DECIMAL(7,1),
  LBXTIB_b SMALLINT DEFAULT 0,
  LBXSLDSI DECIMAL(7,1),
  LBXSLDSI_b SMALLINT DEFAULT 0,
  LBXWBCSI DECIMAL(7,1),
  LBXWBCSI_b SMALLINT DEFAULT 0,
  LBXLYPCT DECIMAL(7,1),
  LBXLYPCT_b SMALLINT DEFAULT 0,
  LBXMOPCT DECIMAL(7,1),
  LBXMOPCT_b SMALLINT DEFAULT 0,
  LBXNEPCT DECIMAL(7,1),
  LBXNEPCT_b SMALLINT DEFAULT 0,
  LBXEOPCT DECIMAL(7,1),
  LBXEOPCT_b SMALLINT DEFAULT 0,
  LBXBAPCT DECIMAL(7,1),
  LBXBAPCT_b SMALLINT DEFAULT 0,
  LBXRBCSI DECIMAL(7,1),
  LBXRBCSI_b SMALLINT DEFAULT 0,
  LBXHGB DECIMAL(7,1),
  LBXHGB_b SMALLINT DEFAULT 0,
  LBXHCT DECIMAL(7,1),
  LBXHCT_b SMALLINT DEFAULT 0,
  LBXMCVSI DECIMAL(7,1),
  LBXMCVSI_b SMALLINT DEFAULT 0,
  LBXMCHSI DECIMAL(7,1),
  LBXMCHSI_b SMALLINT DEFAULT 0,
  LBXMC DECIMAL(7,1),
  LBXMC_b SMALLINT DEFAULT 0,
  LBXRDW DECIMAL(7,1),
  LBXRDW_b SMALLINT DEFAULT 0,
  LBXPLTSI DECIMAL(7,1),
  LBXPLTSI_b SMALLINT DEFAULT 0,
  LBXMPSI DECIMAL(7,1),
  LBXMPSI_b SMALLINT DEFAULT 0,
  PRIMARY KEY(SEQN)
) AS SELECT
  SEQN,RIAGENDR,RIDAGEYR,LBXIRN,0,LBXTIB,0,LBXSLDSI,0,LBXWBCSI,0,LBXLYPCT,0,LBXMOPCT,0,LBXNEPCT,0,LBXEOPCT,0,LBXBAPCT,0,LBXRBCSI,0,LBXHGB,0,LBXHCT,0,LBXMCVSI,0,LBXMCHSI,0,LBXMC,0,LBXRDW,0,LBXPLTSI,0,LBXMPSI,0
FROM CSVREAD('../data/nhanes2005-2006/combined-selected-variables.csv');

SELECT COUNT(*) FROM SurveyB;
SELECT * FROM SurveyB;

## Matrix building

* Each variable is compared with the limits of the NHANES ranges, and the binary _b columns are updated.

In [6]:
-- Computing LBXIRN
UPDATE SurveyB SB
SET SB.LBXIRN_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXIRN' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXIRN<RRb.min);
UPDATE SurveyB SB
SET SB.LBXIRN_b = 1
WHERE SB.LBXIRN_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXIRN' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXIRN>RRb.max);

-- Computing LBXTIB
UPDATE SurveyB SB
SET SB.LBXTIB_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXTIB' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXTIB<RRb.min);
UPDATE SurveyB SB
SET SB.LBXTIB_b = 1
WHERE SB.LBXTIB_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXTIB' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXTIB>RRb.max);

-- Computing LBXSLDSI
UPDATE SurveyB SB
SET SB.LBXSLDSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXSLDSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXSLDSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXSLDSI_b = 1
WHERE SB.LBXSLDSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXSLDSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXSLDSI>RRb.max);

-- Computing LBXWBCSI
UPDATE SurveyB SB
SET SB.LBXWBCSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXWBCSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXWBCSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXWBCSI_b = 1
WHERE SB.LBXWBCSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXWBCSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXWBCSI>RRb.max);

-- Computing LBXLYPCT
UPDATE SurveyB SB
SET SB.LBXLYPCT_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXLYPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXLYPCT<RRb.min);
UPDATE SurveyB SB
SET SB.LBXLYPCT_b =1
WHERE SB.LBXLYPCT_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXLYPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXLYPCT>RRb.max);

-- Computing LBXMOPCT
UPDATE SurveyB SB
SET SB.LBXMOPCT_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMOPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMOPCT<RRb.min);
UPDATE SurveyB SB
SET SB.LBXMOPCT_b = 1
WHERE SB.LBXMOPCT_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMOPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMOPCT>RRb.max);

-- Computing LBXNEPCT
UPDATE SurveyB SB
SET SB.LBXNEPCT_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXNEPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXNEPCT<RRb.min);
UPDATE SurveyB SB
SET SB.LBXNEPCT_b = 1
WHERE SB.LBXNEPCT_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXNEPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXNEPCT>RRb.max);

-- Computing LBXEOPCT
UPDATE SurveyB SB
SET SB.LBXEOPCT_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXEOPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXEOPCT<RRb.min);
UPDATE SurveyB SB
SET SB.LBXEOPCT_b = 1
WHERE SB.LBXEOPCT_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXEOPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXEOPCT>RRb.max);

-- Computing LBXBAPCT
UPDATE SurveyB SB
SET SB.LBXBAPCT_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXBAPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXBAPCT<RRb.min);
UPDATE SurveyB SB
SET SB.LBXBAPCT_b = 1
WHERE SB.LBXBAPCT_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXBAPCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXBAPCT>RRb.max);

-- Computing LBXRBCSI
UPDATE SurveyB SB
SET SB.LBXRBCSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRBCSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXRBCSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXRBCSI_b = 1
WHERE SB.LBXRBCSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRBCSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXRBCSI>RRb.max);

-- Computing LBXHGB
UPDATE SurveyB SB
SET SB.LBXHGB_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHGB' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXHGB<RRb.min);
UPDATE SurveyB SB
SET SB.LBXHGB_b = 1
WHERE SB.LBXHGB_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHGB' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXHGB>RRb.max);

-- Computing LBXHCT
UPDATE SurveyB SB
SET SB.LBXHCT_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXHCT<RRb.min);
UPDATE SurveyB SB
SET SB.LBXHCT_b = 1
WHERE SB.LBXHCT_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHCT' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXHCT>RRb.max);

-- Computing LBXMCVSI
UPDATE SurveyB SB
SET SB.LBXMCVSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCVSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMCVSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXMCVSI_b = 1
WHERE SB.LBXMCVSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCVSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMCVSI>RRb.max);

-- Computing LBXMCHSI
UPDATE SurveyB SB
SET SB.LBXMCHSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCHSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMCHSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXMCHSI_b = 1
WHERE SB.LBXMCHSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCHSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMCHSI>RRb.max);

-- Computing LBXMC
UPDATE SurveyB SB
SET SB.LBXMC_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMC' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMC<RRb.min);
UPDATE SurveyB SB
SET SB.LBXMC_b = 1
WHERE SB.LBXMC_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMC' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMC>RRb.max);

-- Computing LBXRDW
UPDATE SurveyB SB
SET SB.LBXRDW_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRDW' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXRDW<RRb.min);
UPDATE SurveyB SB
SET SB.LBXRDW_b = 1
WHERE SB.LBXRDW_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRDW' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXRDW>RRb.max);

-- Computing LBXPLTSI
UPDATE SurveyB SB
SET SB.LBXPLTSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXPLTSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXPLTSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXPLTSI_b = 1
WHERE SB.LBXPLTSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXPLTSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXPLTSI>RRb.max);

-- Computing LBXMPSI
UPDATE SurveyB SB
SET SB.LBXMPSI_b = 1
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMPSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMPSI<RRb.min);
UPDATE SurveyB SB
SET SB.LBXMPSI_b = 1
WHERE SB.LBXMPSI_b = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMPSI' AND SB.RIAGENDR=RRb.gender AND SB.RIDAGEYR>=RRb.ageStart AND SB.RIDAGEYR<=RRb.ageEnd AND SB.LBXMPSI>RRb.max);

## Final Matrix

* Building of the final matrix that has the identification of the person, a binary _b matrix, and a profile built by the concatenation of lines in the binary matrix.
* The profile represents the in a binary form what is out of the ranges in each person.
* Only anormal persons are filtered.

* The resulting matrix produces a CSV file.

In [7]:
DROP VIEW IF EXISTS DeviationProfiles;
DROP VIEW IF EXISTS CorrelationMatrix;

CREATE VIEW CorrelationMatrix AS
SELECT DISTINCT SB.SEQN, 
  CONCAT(SB.LBXIRN_b, SB.LBXTIB_b, SB.LBXSLDSI_b, SB.LBXWBCSI_b, SB.LBXLYPCT_b, SB.LBXMOPCT_b, SB.LBXNEPCT_b, SB.LBXEOPCT_b, SB.LBXBAPCT_b, SB.LBXRBCSI_b, SB.LBXHGB_b, SB.LBXHCT_b, SB.LBXMCVSI_b, SB.LBXMCHSI_b, SB.LBXMC_b, SB.LBXRDW_b, SB.LBXPLTSI_b, SB.LBXMPSI_b) AS profile,
  SB.LBXIRN_b, SB.LBXTIB_b, SB.LBXSLDSI_b, SB.LBXWBCSI_b, SB.LBXLYPCT_b, SB.LBXMOPCT_b, SB.LBXNEPCT_b, SB.LBXEOPCT_b, SB.LBXBAPCT_b, SB.LBXRBCSI_b, SB.LBXHGB_b, SB.LBXHCT_b, SB.LBXMCVSI_b, SB.LBXMCHSI_b, SB.LBXMC_b, SB.LBXRDW_b, SB.LBXPLTSI_b, SB.LBXMPSI_b
FROM SurveyB SB, ReferenceRanges RR
WHERE SB.RIAGENDR=RR.gender AND SB.RIDAGEYR>=RR.ageStart AND SB.RIDAGEYR<=RR.ageEnd AND
(LBXIRN_b>0 OR LBXTIB_b>0 OR LBXSLDSI_b>0 OR LBXWBCSI_b>0 OR LBXLYPCT_b>0 OR LBXMOPCT_b>0 OR LBXNEPCT_b>0 OR LBXEOPCT_b>0 OR LBXBAPCT_b>0 OR LBXRBCSI_b>0 OR LBXHGB_b>0 OR LBXHCT_b>0 OR LBXMCVSI_b>0 OR LBXMCHSI_b>0 OR LBXMC_b>0 OR LBXRDW_b>0 OR LBXPLTSI_b>0 OR LBXMPSI_b>0);

SELECT COUNT(*) FROM CorrelationMatrix;
SELECT * FROM CorrelationMatrix;

CALL CSVWRITE('../data/nhanes2005-2006/correlation-matrix.csv', 'SELECT * FROM CorrelationMatrix');

# Profiles network

* Persons are here related from their binary profiles, producing a profiles network.

## Grouping profiles

* Profiles are grouped according ro a binary pattern and people with the same profile are aggregated.

In [8]:
DROP VIEW IF EXISTS DeviationProfiles;

CREATE VIEW DeviationProfiles AS
SELECT CM.profile, COUNT(*) AS individuals
FROM CorrelationMatrix CM
GROUP BY CM.profile;

SELECT SUM(individuals) FROM DeviationProfiles;
SELECT * FROM DeviationProfiles;

CALL CSVWRITE('../data/nhanes2005-2006/profile-number-deviation.csv', 'SELECT DP.profile AS id, DP.individuals AS weight FROM DeviationProfiles DP');

# Matrix with deviation intensity

* This second matrix records the deviation of variables that overcomes the limits and how much the overcome.

## Generation of the starting matrix initialized with 0

In [9]:
DROP TABLE IF EXISTS SurveyD;
CREATE TABLE SurveyD (
  SEQN VARCHAR(8),
  RIAGENDR VARCHAR(1),
  RIDAGEYR SMALLINT,
  LBXIRN DECIMAL(7,1),
  LBXIRN_d DECIMAL(7,1) DEFAULT 0,
  LBXTIB DECIMAL(7,1),
  LBXTIB_d DECIMAL(7,1) DEFAULT 0,
  LBXSLDSI DECIMAL(7,1),
  LBXSLDSI_d DECIMAL(7,1) DEFAULT 0,
  LBXWBCSI DECIMAL(7,1),
  LBXWBCSI_d DECIMAL(7,1) DEFAULT 0,
  LBXLYPCT DECIMAL(7,1),
  LBXLYPCT_d DECIMAL(7,1) DEFAULT 0,
  LBXMOPCT DECIMAL(7,1),
  LBXMOPCT_d DECIMAL(7,1) DEFAULT 0,
  LBXNEPCT DECIMAL(7,1),
  LBXNEPCT_d DECIMAL(7,1) DEFAULT 0,
  LBXEOPCT DECIMAL(7,1),
  LBXEOPCT_d DECIMAL(7,1) DEFAULT 0,
  LBXBAPCT DECIMAL(7,1),
  LBXBAPCT_d DECIMAL(7,1) DEFAULT 0,
  LBXRBCSI DECIMAL(7,1),
  LBXRBCSI_d DECIMAL(7,1) DEFAULT 0,
  LBXHGB DECIMAL(7,1),
  LBXHGB_d DECIMAL(7,1) DEFAULT 0,
  LBXHCT DECIMAL(7,1),
  LBXHCT_d DECIMAL(7,1) DEFAULT 0,
  LBXMCVSI DECIMAL(7,1),
  LBXMCVSI_d DECIMAL(7,1) DEFAULT 0,
  LBXMCHSI DECIMAL(7,1),
  LBXMCHSI_d DECIMAL(7,1) DEFAULT 0,
  LBXMC DECIMAL(7,1),
  LBXMC_d DECIMAL(7,1) DEFAULT 0,
  LBXRDW DECIMAL(7,1),
  LBXRDW_d DECIMAL(7,1) DEFAULT 0,
  LBXPLTSI DECIMAL(7,1),
  LBXPLTSI_d DECIMAL(7,1) DEFAULT 0,
  LBXMPSI DECIMAL(7,1),
  LBXMPSI_d DECIMAL(7,1) DEFAULT 0,
  PRIMARY KEY(SEQN)
) AS SELECT
  SEQN,RIAGENDR,RIDAGEYR,LBXIRN,0,LBXTIB,0,LBXSLDSI,0,LBXWBCSI,0,LBXLYPCT,0,LBXMOPCT,0,LBXNEPCT,0,LBXEOPCT,0,LBXBAPCT,0,LBXRBCSI,0,LBXHGB,0,LBXHCT,0,LBXMCVSI,0,LBXMCHSI,0,LBXMC,0,LBXRDW,0,LBXPLTSI,0,LBXMPSI,0
FROM CSVREAD('../data/nhanes2005-2006/combined-selected-variables.csv');

## Matrix building

* Each variable is compared with the limits of the NHANES ranges, and the deviation _d columns receive the difference.

In [10]:
-- Computing LBXIRN
UPDATE SurveyD SD
SET SD.LBXIRN_d =
(SELECT RRa.min-SD.LBXIRN
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXIRN' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXIRN<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXIRN' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXIRN<RRb.min);
UPDATE SurveyD SD
SET SD.LBXIRN_d =
(SELECT SD.LBXIRN-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXIRN' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXIRN>RRa.max)
WHERE SD.LBXIRN_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXIRN' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXIRN>RRb.max);

-- Computing LBXTIB
UPDATE SurveyD SD
SET SD.LBXTIB_d =
(SELECT RRa.min-SD.LBXTIB
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXTIB' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXTIB<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXTIB' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXTIB<RRb.min);
UPDATE SurveyD SD
SET SD.LBXTIB_d =
(SELECT SD.LBXTIB-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXTIB' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXTIB>RRa.max)
WHERE SD.LBXTIB_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXTIB' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXTIB>RRb.max);

-- Computing LBXSLDSI
UPDATE SurveyD SD
SET SD.LBXSLDSI_d =
(SELECT RRa.min-SD.LBXSLDSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXSLDSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXSLDSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXSLDSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXSLDSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXSLDSI_d =
(SELECT SD.LBXSLDSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXSLDSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXSLDSI>RRa.max)
WHERE SD.LBXSLDSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXSLDSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXSLDSI>RRb.max);

-- Computing LBXWBCSI
UPDATE SurveyD SD
SET SD.LBXWBCSI_d =
(SELECT RRa.min-SD.LBXWBCSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXWBCSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXWBCSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXWBCSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXWBCSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXWBCSI_d =
(SELECT SD.LBXWBCSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXWBCSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXWBCSI>RRa.max)
WHERE SD.LBXWBCSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXWBCSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXWBCSI>RRb.max);

-- Computing LBXLYPCT
UPDATE SurveyD SD
SET SD.LBXLYPCT_d =
(SELECT RRa.min-SD.LBXLYPCT
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXLYPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXLYPCT<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXLYPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXLYPCT<RRb.min);
UPDATE SurveyD SD
SET SD.LBXLYPCT_d =
(SELECT SD.LBXLYPCT-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXLYPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXLYPCT>RRa.max)
WHERE SD.LBXLYPCT_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXLYPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXLYPCT>RRb.max);

-- Computing LBXMOPCT
UPDATE SurveyD SD
SET SD.LBXMOPCT_d =
(SELECT RRa.min-SD.LBXMOPCT
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMOPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMOPCT<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMOPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMOPCT<RRb.min);
UPDATE SurveyD SD
SET SD.LBXMOPCT_d =
(SELECT SD.LBXMOPCT-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMOPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMOPCT>RRa.max)
WHERE SD.LBXMOPCT_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMOPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMOPCT>RRb.max);

-- Computing LBXNEPCT
UPDATE SurveyD SD
SET SD.LBXNEPCT_d =
(SELECT RRa.min-SD.LBXNEPCT
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXNEPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXNEPCT<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXNEPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXNEPCT<RRb.min);
UPDATE SurveyD SD
SET SD.LBXNEPCT_d =
(SELECT SD.LBXNEPCT-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXNEPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXNEPCT>RRa.max)
WHERE SD.LBXNEPCT_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXNEPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXNEPCT>RRb.max);

-- Computing LBXEOPCT
UPDATE SurveyD SD
SET SD.LBXEOPCT_d =
(SELECT RRa.min-SD.LBXEOPCT
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXEOPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXEOPCT<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXEOPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXEOPCT<RRb.min);
UPDATE SurveyD SD
SET SD.LBXEOPCT_d =
(SELECT SD.LBXEOPCT-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXEOPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXEOPCT>RRa.max)
WHERE SD.LBXEOPCT_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXEOPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXEOPCT>RRb.max);

-- Computing LBXBAPCT
UPDATE SurveyD SD
SET SD.LBXBAPCT_d =
(SELECT RRa.min-SD.LBXBAPCT
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXBAPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXBAPCT<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXBAPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXBAPCT<RRb.min);
UPDATE SurveyD SD
SET SD.LBXBAPCT_d =
(SELECT SD.LBXBAPCT-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXBAPCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXBAPCT>RRa.max)
WHERE SD.LBXBAPCT_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXBAPCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXBAPCT>RRb.max);

-- Computing LBXRBCSI
UPDATE SurveyD SD
SET SD.LBXRBCSI_d =
(SELECT RRa.min-SD.LBXRBCSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXRBCSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXRBCSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRBCSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXRBCSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXRBCSI_d =
(SELECT SD.LBXRBCSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXRBCSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXRBCSI>RRa.max)
WHERE SD.LBXRBCSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRBCSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXRBCSI>RRb.max);

-- Computing LBXHGB
UPDATE SurveyD SD
SET SD.LBXHGB_d =
(SELECT RRa.min-SD.LBXHGB
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXHGB' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXHGB<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHGB' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXHGB<RRb.min);
UPDATE SurveyD SD
SET SD.LBXHGB_d =
(SELECT SD.LBXHGB-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXHGB' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXHGB>RRa.max)
WHERE SD.LBXHGB_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHGB' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXHGB>RRb.max);

-- Computing LBXHCT
UPDATE SurveyD SD
SET SD.LBXHCT_d =
(SELECT RRa.min-SD.LBXHCT
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXHCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXHCT<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXHCT<RRb.min);
UPDATE SurveyD SD
SET SD.LBXHCT_d =
(SELECT SD.LBXHCT-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXHCT' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXHCT>RRa.max)
WHERE SD.LBXHCT_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXHCT' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXHCT>RRb.max);

-- Computing LBXMCVSI
UPDATE SurveyD SD
SET SD.LBXMCVSI_d =
(SELECT RRa.min-SD.LBXMCVSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMCVSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMCVSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCVSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMCVSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXMCVSI_d =
(SELECT SD.LBXMCVSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMCVSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMCVSI>RRa.max)
WHERE SD.LBXMCVSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCVSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMCVSI>RRb.max);

-- Computing LBXMCHSI
UPDATE SurveyD SD
SET SD.LBXMCHSI_d =
(SELECT RRa.min-SD.LBXMCHSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMCHSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMCHSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCHSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMCHSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXMCHSI_d =
(SELECT SD.LBXMCHSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMCHSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMCHSI>RRa.max)
WHERE SD.LBXMCHSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMCHSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMCHSI>RRb.max);

-- Computing LBXMC
UPDATE SurveyD SD
SET SD.LBXMC_d =
(SELECT RRa.min-SD.LBXMC
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMC' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMC<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMC' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMC<RRb.min);
UPDATE SurveyD SD
SET SD.LBXMC_d =
(SELECT SD.LBXMC-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMC' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMC>RRa.max)
WHERE SD.LBXMC_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMC' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMC>RRb.max);

-- Computing LBXRDW
UPDATE SurveyD SD
SET SD.LBXRDW_d =
(SELECT RRa.min-SD.LBXRDW
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXRDW' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXRDW<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRDW' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXRDW<RRb.min);
UPDATE SurveyD SD
SET SD.LBXRDW_d =
(SELECT SD.LBXRDW-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXRDW' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXRDW>RRa.max)
WHERE SD.LBXRDW_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXRDW' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXRDW>RRb.max);

-- Computing LBXPLTSI
UPDATE SurveyD SD
SET SD.LBXPLTSI_d =
(SELECT RRa.min-SD.LBXPLTSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXPLTSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXPLTSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXPLTSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXPLTSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXPLTSI_d =
(SELECT SD.LBXPLTSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXPLTSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXPLTSI>RRa.max)
WHERE SD.LBXPLTSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXPLTSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXPLTSI>RRb.max);

-- Computing LBXMPSI
UPDATE SurveyD SD
SET SD.LBXMPSI_d =
(SELECT RRa.min-SD.LBXMPSI
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMPSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMPSI<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMPSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMPSI<RRb.min);
UPDATE SurveyD SD
SET SD.LBXMPSI_d =
(SELECT SD.LBXMPSI-RRa.max
 FROM ReferenceRanges RRa
 WHERE RRa.variable='LBXMPSI' AND SD.RIAGENDR=RRa.gender AND SD.RIDAGEYR>=RRa.ageStart AND SD.RIDAGEYR<=RRa.ageEnd AND SD.LBXMPSI>RRa.max)
WHERE SD.LBXMPSI_d = 0 AND
EXISTS (SELECT RRb.max
 FROM ReferenceRanges RRb
 WHERE RRb.variable='LBXMPSI' AND SD.RIAGENDR=RRb.gender AND SD.RIDAGEYR>=RRb.ageStart AND SD.RIDAGEYR<=RRb.ageEnd AND SD.LBXMPSI>RRb.max);

## Final Matrix

* Building of the final matrix that has the identification of the person and a deviation _d matrix.
* Only anormal persons are filtered.

In [11]:
DROP VIEW IF EXISTS CorrelationMatrixWeighted;

CREATE VIEW CorrelationMatrixWeighted AS
SELECT DISTINCT SD.SEQN, SD.LBXIRN_d, SD.LBXTIB_d, SD.LBXSLDSI_d, SD.LBXWBCSI_d, SD.LBXLYPCT_d, SD.LBXMOPCT_d, SD.LBXNEPCT_d, SD.LBXEOPCT_d, SD.LBXBAPCT_d, SD.LBXRBCSI_d, SD.LBXHGB_d, SD.LBXHCT_d, SD.LBXMCVSI_d, SD.LBXMCHSI_d, SD.LBXMC_d, SD.LBXRDW_d, SD.LBXPLTSI_d, SD.LBXMPSI_d
FROM SurveyD SD, ReferenceRanges RR
WHERE SD.RIAGENDR=RR.gender AND SD.RIDAGEYR>=RR.ageStart AND SD.RIDAGEYR<=RR.ageEnd AND
(LBXIRN_d>0 OR LBXTIB_d>0 OR LBXSLDSI_d>0 OR LBXWBCSI_d>0 OR LBXLYPCT_d>0 OR LBXMOPCT_d>0 OR LBXNEPCT_d>0 OR LBXEOPCT_d>0 OR LBXBAPCT_d>0 OR LBXRBCSI_d>0 OR LBXHGB_d>0 OR LBXHCT_d>0 OR LBXMCVSI_d>0 OR LBXMCHSI_d>0 OR LBXMC_d>0 OR LBXRDW_d>0 OR LBXPLTSI_d>0 OR LBXMPSI_d>0);

SELECT COUNT(*) FROM CorrelationMatrixWeighted;
SELECT * FROM CorrelationMatrixWeighted;

CALL CSVWRITE('../data/nhanes2005-2006/correlation-matrix-weighted.csv', 'SELECT * FROM CorrelationMatrixWeighted');

# Variables Network

* In this network each node is a variable and each edge indicates that two variables are correlated in a certain intensity.

## List of the variable pairs

* This view prepares the list of correlation pairs initialized with 0.

In [12]:
DROP VIEW IF EXISTS VariablesCorrelation;
DROP VIEW IF EXISTS Variables;

CREATE VIEW Variables AS
SELECT DISTINCT variable AS var1 FROM ReferenceRanges;

CREATE VIEW VariablesCorrelation AS
SELECT DISTINCT Variables.var1, ReferenceRanges.variable AS var2, 0 AS correlation
FROM Variables, ReferenceRanges
WHERE Variables.var1 < ReferenceRanges.variable;

## Survey verticalization

* Persons and variables that are originally presented as a matrix are transformed in a list: person, variable and value. This list will facilitate the subsequent analyses.

In [13]:
DROP VIEW IF EXISTS VerticalSurvey;

CREATE VIEW VerticalSurvey AS
  SELECT SU.SEQN, RR.variable, SU.LBXIRN AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXIRN'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXTIB AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXTIB'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXSLDSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXSLDSI'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXWBCSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXWBCSI'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXLYPCT AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXLYPCT'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXMOPCT AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXMOPCT'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXNEPCT AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXNEPCT'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXEOPCT AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXEOPCT'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXBAPCT AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXBAPCT'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXRBCSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXRBCSI'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXHGB AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXHGB'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXHCT AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXHCT'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXMCVSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXMCVSI'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXMCHSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXMCHSI'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXMC AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXMC'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXRDW AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXRDW'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXPLTSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXPLTSI'
UNION
  SELECT SU.SEQN, RR.variable, SU.LBXMPSI AS value, 0 AS deviation
  FROM Survey SU, ReferenceRanges RR
  WHERE RR.variable='LBXMPSI'
;

-- transformation of the view in a table to enable updates
DROP TABLE IF EXISTS VerticalSurveyD;
CREATE TABLE VerticalSurveyD (
  SEQN VARCHAR(8),
  variable VARCHAR(8),
  value DECIMAL(7,1),
  deviation DECIMAL(7,1),
  PRIMARY KEY(SEQN, variable)
) AS SELECT * FROM VerticalSurvey;

CALL CSVWRITE('../data/nhanes2005-2006/vertical-survey.csv', 'SELECT SEQN,variable,value FROM VerticalSurvey');

47970

## Computation of the deviation value for the variables that are out of the limits

In [14]:
UPDATE VerticalSurveyD VS
SET VS.deviation =
(SELECT RRa.min-VS.value
 FROM Survey SUa, ReferenceRanges RRa
 WHERE RRa.variable=VS.variable AND SUa.SEQN=VS.SEQN AND SUa.RIAGENDR=RRa.gender AND SUa.RIDAGEYR>=RRa.ageStart AND SUa.RIDAGEYR<=RRa.ageEnd AND VS.value<RRa.min)
WHERE EXISTS
(SELECT RRb.min
 FROM Survey SUb, ReferenceRanges RRb
 WHERE RRb.variable=VS.variable AND SUb.SEQN=VS.SEQN AND SUb.RIAGENDR=RRb.gender AND SUb.RIDAGEYR>=RRb.ageStart AND SUb.RIDAGEYR<=RRb.ageEnd AND VS.value<RRb.min);

UPDATE VerticalSurveyD VS
SET VS.deviation =
(SELECT VS.value-RRa.max
 FROM Survey SUa, ReferenceRanges RRa
 WHERE RRa.variable=VS.variable AND SUa.SEQN=VS.SEQN AND SUa.RIAGENDR=RRa.gender AND SUa.RIDAGEYR>=RRa.ageStart AND SUa.RIDAGEYR<=RRa.ageEnd AND VS.value>RRa.max)
WHERE EXISTS
(SELECT RRb.max
 FROM Survey SUb, ReferenceRanges RRb
 WHERE RRb.variable=VS.variable AND SUb.SEQN=VS.SEQN AND SUb.RIAGENDR=RRb.gender AND SUb.RIDAGEYR>=RRb.ageStart AND SUb.RIDAGEYR<=RRb.ageEnd AND VS.value>RRb.max);
 
SELECT * FROM VerticalSurveyD WHERE deviation > 0;

CALL CSVWRITE('../data/nhanes2005-2006/vertical-survey-deviation.csv', 'SELECT * FROM VerticalSurveyD WHERE deviation > 0');

## Number of abnormalities by variable

In [15]:
SELECT variable as id, COUNT(*) as weight FROM VerticalSurveyD VS WHERE deviation>0 GROUP BY variable;

CALL CSVWRITE('../data/nhanes2005-2006/variable-number-deviation.csv', 'SELECT variable as id, COUNT(*) as weight FROM VerticalSurveyD VS WHERE deviation>0 GROUP BY variable');

## Variable correlation by person

* Pairwise correlation of variables that cooccur in the same person.

In [16]:
DROP VIEW IF EXISTS VariablePairCorrelation;
DROP VIEW IF EXISTS IndividualVariablesCorrelation;

CREATE VIEW IndividualVariablesCorrelation AS
SELECT VS1.SEQN, CM.profile, VC.var1, VC.var2
FROM VariablesCorrelation VC, VerticalSurveyD VS1, VerticalSurveyD VS2, CorrelationMatrix CM
WHERE VS1.SEQN = VS2.SEQN AND VS1.variable = VC.var1 AND VS2.variable = VC.var2 AND 
      VS1.deviation > 0 AND VS2.deviation > 0 AND
      VS1.SEQN = CM.SEQN;

SELECT * FROM IndividualVariablesCorrelation
ORDER BY var1, var2;

## Correlation of variable pairs

* Aggregation of correlations of variable pairs.
* Preparation to build a network where variables are vertices and edges connect variables that surpassed the limits together for the same person.

In [17]:
DROP VIEW IF EXISTS VariablePairCorrelation;
CREATE VIEW VariablePairCorrelation AS
SELECT var1 AS source, var2 as TARGET, COUNT(*) AS weight
FROM IndividualVariablesCorrelation
GROUP BY var1, var2;

SELECT * FROM VariablePairCorrelation;

CALL CSVWRITE('../data/nhanes2005-2006/variable-pair-correlation.csv', 'SELECT * FROM VariablePairCorrelation');

# Exercise

Import the file previously created `/data/nhanes2005-2006/variable-pair-correlation.csv` in the Gephi. Which analyses can you do? Insert in the cell below an image of your analyses and a brief description of the result.

## Finishing Part 1

* This notebook is divided into three parts due to memory constraints. The next part is the notebook `sql-network-02-nhanes-complete-p2`.